In [9]:
from selenium.webdriver import Chrome                                   #主要爬蟲工具
from selenium.webdriver.common.keys import Keys                         #需要使用鍵盤功能
from selenium.common.exceptions import NoSuchElementException           #使用try:except除錯
from selenium.common.exceptions import ElementNotInteractableException  #同上
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd                                                     #轉為dataframe
import time                                                             #sleep 減少被發現機率
import random

In [ ]:
driver = Chrome('chromedriver.exe')  
#使用Chrome瀏覽器
ENTER = u'\ue007'                    
#啟用ENTER鍵
url = 'https://www.google.com/search?q=uptodate'
#目標網站
driver.get(url)
                 
driver.find_element_by_xpath('//*[@id="rso"]/div[1]/div/table/tbody/tr[3]/td[1]/div/span/h3/a').click()
#xpath:html內容的絕對路徑
driver.find_element_by_id('userName').send_keys("*******")
driver.find_element_by_id('password').send_keys("*******")
#請輸入帳號密碼登入
time.sleep(5)
#休息5秒
driver.find_element_by_id('btnLoginSubmit').click()
#點擊登入

aurl = 'https://www.uptodate.com/drug-interactions/?source=responsive_home#di-druglist'
#更換網址以切換頁面
driver.get(aurl)

In [ ]:
#輸入藥物名稱並點擊加入清單，使用TRY:EXCEPT避免搜尋不到時跳出錯誤
def drug(a,b):
    for i in range(a,b):
        aurl = 'https://www.uptodate.com/drug-interactions/?source=responsive_home#di-druglist'

        driver.get(aurl)


        driver.find_element_by_xpath('//*[@id="druglist-add"]').click()

        driver.find_element_by_xpath('//*[@id="search-field"]/input').send_keys("{}".format(soseat[i]))

        time.sleep(1)
        try:
            driver.find_element_by_xpath('//*[@id="main-container"]/div[3]/ul/li[1]/a').click()
        except NoSuchElementException:
            driver.find_element_by_xpath('//*[@id="search-back"]').click()
    time.sleep(random.randint(5,15))
    
    searchitem()
    
    driver.find_element_by_xpath('//*[@id="toolbar-left"]').click()
    driver.find_element_by_xpath('//*[@id="toolbar-left"]/a[2]').click()



In [41]:
def searchresults():
    time.sleep(3)
    count = driver.find_element_by_xpath('//*[@id="toolbar-middle"]/span[3]').text.rstrip(' Results')
    if count != '':
        count = int(count)
        for results in range(1,count+1):
            #print(results)
            time.sleep(3)
            driver.find_element_by_xpath('//*[@id="main-container"]/div[5]/ul/li[{}]'.format(results)).click()
            datacreater()

            driver.find_element_by_xpath('//*[@id="toolbar-left"]/a[5]').click()
    else:
        time.sleep(2)

In [ ]:
#由於無法確認藥物數量，持續迴圈直到最後一個藥品時自動停止

def searchitem():
    i = 1
    while True:
        time.sleep(3)
        try:
            driver.find_element_by_xpath('//*[@id="main-container"]/div[4]/ul[1]/li[{}]'.format(i)).click()
        except NoSuchElementException:
            break
        searchresults()
        driver.find_element_by_xpath('//*[@id="analyze-back"]').click()
        i += 1

In [ ]:
def searchresults():
    time.sleep(3)
    count = driver.find_element_by_xpath('//*[@id="toolbar-middle"]/span[3]').text.rstrip(' Results')
    if count != '':
        count = int(count)
        for results in range(1,count+1):
            time.sleep(3)
            try:
                driver.find_element_by_xpath('//*[@id="main-container"]/div[5]/ul/li[{}]'.format(results)).click()
            except ElementClickInterceptedException:
                driver.find_element_by_xpath('//*[@id="main-container"]/div[5]/ul/li[{}]'.format(results+1)).click()
            datacreater()

            driver.find_element_by_xpath('//*[@id="toolbar-left"]/a[5]').click()
    else:
        time.sleep(2)

In [ ]:
def datacreater():
    time.sleep(3)
    
    drugtable = driver.find_element_by_xpath('//*[@id="docContents"]').text
    
    if 'Dependencies' in drugtable:

        Title = driver.find_element_by_xpath('//*[@id="docContents"]/p[1]').text
        Title = Title.lstrip('Title ')
        
        Dependencies = driver.find_element_by_xpath('//*[@id="docContents"]/ul/li/p').text

        RiskRating = driver.find_element_by_xpath('//*[@id="docContents"]/p[4]').text
        RiskRating = RiskRating.lstrip('Risk Rating ')

        AllSummary = driver.find_element_by_xpath('//*[@id="docContents"]/p[5]').text

        Summary =  AllSummary.split('Summary ')[1].split(' Severity ')[0]

        Severity = AllSummary.split('Summary ')[1].split(' Severity ')[1].split(' Reliability Rating ')[0]

        ReliabilityRating = AllSummary.split('Summary ')[1].split(' Severity ')[1].split(' Reliability Rating ')[1]


        PatientManagement = driver.find_element_by_xpath('//*[@id="docContents"]/p[6]').text
        PatientManagement = PatientManagement.lstrip('Patient Management ')

        Discussion = driver.find_element_by_xpath('//*[@id="docContents"]/p[7]').text
        Discussion = Discussion.lstrip('Discussion ')

    df.append([Title,Dependencies,RiskRating,Summary,Severity,ReliabilityRating,PatientManagement,Discussion])